First, we manage all imports for the code.

In [ ]:
#imports
import pandas as pd
import numpy as np
import random
import ast
from torch import nn
import torch
from torch.utils.data import random_split, Subset, DataLoader

import zipfile
import os
import shutil
import time
import logging

Next we process the zipped data to csv. It is stored in .zip, because it is smaller on git and then locally it is extracted to csv.

In [ ]:
with zipfile.ZipFile("data/dataset.zip", "r") as zip_ref:
    zip_ref.extractall("data")

for file_name in os.listdir("data/dataset"):
    source = "data/dataset/" + file_name
    destination = "data/" + file_name
    if os.path.isfile(source): shutil.move(source, destination)

os.rmdir("data/dataset")


# Initialization of different parts
if not os.path.exists("log"): os.mkdir("log")
if os.path.exists("log/training.log"): os.remove("log/training.log")
logging.basicConfig(filename="log/training.log", encoding="utf-8", level=logging.DEBUG)
del source, destination

In [ ]:
device = torch.device('cpu') #torch.device('cuda' if torch.cuda.is_available() else 'cpu') For faster training in the end

class SongModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SongModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = input_size * 3
        self.output_size = output_size
        self.layer_1 =   nn.Linear(self.input_size, self.hidden_size)
        self.layer_2 =   nn.Linear(self.hidden_size, self.hidden_size)
        self.layer_out = nn.Linear(self.hidden_size, self.output_size)
        self.sigmoid =   nn.Sigmoid()
        self.tanh =      nn.Tanh()
        self.softmax =   nn.Softmax()

        nn.init.xavier_uniform_(self.layer_1.weight)
        nn.init.zeros_(self.layer_1.bias)
        nn.init.xavier_uniform_(self.layer_2.weight)
        nn.init.zeros_(self.layer_2.bias)
        nn.init.xavier_uniform_(self.layer_out.weight)
        nn.init.zeros_(self.layer_out.bias)


    def forward(self, model_input):
        x = self.sigmoid(self.layer_1(model_input))
        x = self.tanh(self.layer_2(x))
        x = self.softmax(self.layer_out(x))
        return x

model = SongModel(1, 3).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.025)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    max_iterations = 5000
    dataloader_size = len(dataloader)
    for iteration in range(max_iterations):
        total_loss = 0
        sample_iteration = 0
        for sample in dataloader:
            model_input = sample[0]
            should = sample[1]
            predict = model(model_input)
            loss = loss_fn(predict, should)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss
            sample_iteration += 1
            print(f"Iteration: {iteration+1:4}, Iteration progress: {sample_iteration * 100 / dataloader_size:3.2f}%", end="\r")

        logging.info(f"Finish iteration {iteration+1:4}, Average loss: {total_loss / dataloader_size}")
    return

In [ ]:
def save_model(model_s: SongModel, path) -> None:
    torch.save(model_s.state_dict(), path)


def load_model(model_l: SongModel, path) -> SongModel:
    model_l.load_state_dict(torch.load(path))
    model_l.eval()
    return model_l